# Movie Genre Classification Based on Trailer Barcode

In [1]:
from __future__ import print_function

In [2]:
# We'll need numpy for some mathematical operations
import numpy as np


# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline


# and IPython.display for audio output
import IPython.display


# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display
import glob
import os

import tensorflow as tf
from matplotlib.pyplot import specgram
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [3]:
#Run on GPU
#import theano
#theano.config.device = 'gpu'
#theano.config.floatX = 'float32'

## Movie BarCode

### Loading movie barcodes from directory

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from PIL import Image
import PIL
from keras import backend as K
K.set_image_dim_ordering('th')

def parse_movie_bar_codes_for_cnn(parent_dir,sub_dirs, width, height, file_ext="*.png"):
    movie_barcodes = []        
    labels = []
    number = 0
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):            
            label = fn.split('/')[2].split('.')[0] #<path>/<label>.<title>.png
            title = fn.split('/')[2].split('.')[1] #<path>/<label>.<title>.png
            number += 1 
            print("label {} title {} number {}".format(label, title, number))
            try:
                image = Image.open(fn)            
                print("Creating numpy representation of image %s", fn)
                resize = image.resize((width, height), PIL.Image.ANTIALIAS) 
                resize.load()
                data = np.asarray( resize, dtype="uint8" )
                print(data.shape)
                movie_barcodes.append(data)
                labels.append(label)
            except Exception as e:
                print("Error {} encountered while parsing file: {}".format(e,fn))
                continue
    
    #reshape to tensor
    movie_barcodes = np.asarray(movie_barcodes).reshape(len(movie_barcodes),width, height,3)
   
    print("features size {}".format(np.array(movie_barcodes).shape))
    print("labels size {}".format(np.array(labels).shape))
    return np.array(movie_barcodes), np.array(labels)

In [5]:
parent_dir = "movie_genres"
sub_dirs = ["trailers"]
width = 125
heigth = 25
features, labels = parse_movie_bar_codes_for_cnn(parent_dir,sub_dirs,width, heigth)

label drama title Fight Club Trailer - HD number 1
Creating numpy representation of image %s movie_genres/trailers/drama.Fight Club Trailer - HD.f136.mp4.png
(25, 125, 3)
label action title Teen Wolf Official Trailer #1 - Michael J number 2
Creating numpy representation of image %s movie_genres/trailers/action.Teen Wolf Official Trailer #1 - Michael J. Fox Movie (1985) HD.f134.mp4.png
(25, 125, 3)
label action title Angel Of The City - Cobra Soundtrack number 3
Creating numpy representation of image %s movie_genres/trailers/action.Angel Of The City - Cobra Soundtrack.f135.mp4.png
(25, 125, 3)
label horror title Hellraiser (1987) Trailer number 4
Creating numpy representation of image %s movie_genres/trailers/horror.Hellraiser (1987) Trailer.f134.mp4.png
(25, 125, 3)
label dance title Flashdance (1983) Trailer number 5
Creating numpy representation of image %s movie_genres/trailers/dance.Flashdance (1983) Trailer.f135.mp4.png
(25, 125, 3)
label comedy title The Mask Trailer number 6
Cre

In [6]:
## ML Preprocessing

In [7]:
le = preprocessing.LabelEncoder()
le.fit(labels)
print(list(le.classes_))
labels_encoded = le.transform(labels) 
print(labels_encoded)

['Drama', 'action', 'adventure', 'comedy', 'crime', 'dance', 'documentary', 'drama', 'gangster', 'horror', 'musical', 'romance', 'scifi', 'war', 'western']
[ 7  1  1  9  5  3  2  1  9  5  2  1 11  3  3  2 12  1  5  2  5  7  7  7  8
  8  9 11 11 12 12 13 13 13 14 14 14  9  7  3  9  7  9  7  7 10  9  1  9  9
  3 12  1  3  1  3  1  9 13 13 10  1  1  9  9  7  9  9  4  9  2  3  1  7  4
  9  9  7  9  7  7  9  1  9  3  9  9  1  1  1 10  9  7  9  1  2  1  3  2 12
  7  7  1  2  1  9  2  1  7  9  1 12  7  9  9  2  2  2  2  2  2  2  7  7  7
  4  1  3  1  1  1  1  1  1  1  1  1  1  1  1  3 12  1 12  1  9  1  9  1  9
  9  9  9  7  9  9  9  9  9  2  9  9  9  9  9  7  3  9  9  0  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  1  1  7  1  7  9  1  1  1
  1  1  1  2  1  7  1  2 10  9  0  1  1 12  1  1  7  7  1  1  2  1  2  1  2
  1  3  1  4  1  1  1  1  1  1  1  7  9  2 12  1  2  1  3  3  3  1  3  9 11
  7  7  2  2  2  9  1  4  2 12  2  1  1  7  1 12  2  1  3  1  7 12  3 12 12
 12 12 1

# CNN training

A stacked convolutional neural network (CNN) to classify dataset. Uses Tensorflow, with Keras to provide some 

### Separar os dados (treino, validação e teste) 
70% para treino e 30% para testes.

Dos 70% de treino (deixar 80% para treino e 20% para validaçao)

In [8]:
from sklearn.model_selection import train_test_split
X = features.astype('float32')
X /= 255 #normalize to 1
y = labels_encoded

X_train, test_x, y_train, test_y = train_test_split(X, y, test_size=0.3) #, stratify=y
train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.2) #,stratify=y_train

train_y = train_y.reshape((-1, 1))
valid_y = valid_y.reshape((-1, 1))
test_y = test_y.reshape((-1, 1))

print("TRAIN x size {} y size {}".format(train_x.shape, train_y.shape))
print("VALIDATION x size {} y size {}".format(valid_x.shape, valid_y.shape))
print("TEST x size {} y size {}".format(test_x.shape, test_y.shape))

#print("TEST y {}".format(test_y))

TRAIN x size (309, 125, 25, 3) y size (309, 1)
VALIDATION x size (78, 125, 25, 3) y size (78, 1)
TEST x size (166, 125, 25, 3) y size (166, 1)


### Create CNN Architecture

In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, TensorBoard, RemoteMonitor, EarlyStopping
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit
from keras import backend as K
from tensorflow.python.ops import control_flow_ops
import tensorflow as tf
from keras.optimizers import SGD
from keras.regularizers import l2, activity_l2
from keras.utils import np_utils
from sklearn import metrics 
tf.python.control_flow_ops = tf
K.set_image_dim_ordering('tf')

# data dimension parameters 
num_channels = 3
num_labels = len(list(le.classes_))

f_size = 3

model = Sequential()

model.add(Convolution2D(128, f_size, f_size, border_mode='same', input_shape=(width, heigth, num_channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Convolution2D(64, f_size, f_size, border_mode='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Convolution2D(32, f_size, f_size, border_mode='valid'))
model.add(Activation('relu'))

# flatten output into a single dimension, let Keras do shape inference
model.add(Flatten())

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5)) 

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5)) 

model.add(Dense(num_labels, W_regularizer=l2(0.001)))
model.add(Activation('softmax'))

# create a SGD optimiser
sgd = SGD(lr=0.001, momentum=0.9, decay=5e-4, nesterov=True)

tensorboard = TensorBoard(log_dir='tensorflow/logs', histogram_freq=0, write_graph=True, write_images=True)
checkpointer = ModelCheckpoint(filepath='tensorflow/tmp/weigths_movie_barcode_classifier.hdf5', verbose=1, save_best_only=True)
#remote = RemoteMonitor(root='http://localhost:9001')
earlystop = EarlyStopping(monitor="val_loss", patience=30, verbose=0)
# now compile the model, Keras will take care of the Tensorflow/Theano boilerplate
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

/home/paulofaria/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### Select best model (using Validation)

In [10]:
#model = KerasClassifier(build_fn=create_model,verbose=0)
#nb_epochs_range = np.linspace(80, 90, num=2, dtype=int)
#batch_size_range = np.logspace(6.0, 7.0, base=2, num=2, dtype=int)
#param_grid = dict(nb_epoch=nb_epochs_range, batch_size=batch_size_range)
#validation = StratifiedShuffleSplit(train_y, n_iter=1, test_size=0.2)
#grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=validation)
#grid_result = grid.fit(train_x, train_y)

#print("Os melhores parametros para Cnn params %s com um score de %0.2f"
#     % (grid.best_params_, grid.best_score_))        


### Train best model

In [11]:
#%timeit model.fit(train_x, train_y, validation_data=(valid_x, valid_y), batch_size=grid.best_params_['batch_size'], nb_epoch=grid.best_params_['nb_epoch'], verbose=0)
#%timeit model.fit(train_x, train_y, validation_data=(valid_x, valid_y), batch_size=80, nb_epoch=128, verbose=0)
model.fit(train_x, train_y, validation_data=(valid_x, valid_y), batch_size=80, nb_epoch=128, verbose=0, callbacks=[earlystop,checkpointer,tensorboard])

Instructions for updating:
Please switch to tf.summary.merge_all.
Epoch 00000: val_loss improved from inf to 2.72532, saving model to tensorflow/tmp/weigths_movie_barcode_classifier.hdf5
Epoch 00001: val_loss improved from 2.72532 to 2.71458, saving model to tensorflow/tmp/weigths_movie_barcode_classifier.hdf5
Epoch 00002: val_loss improved from 2.71458 to 2.69877, saving model to tensorflow/tmp/weigths_movie_barcode_classifier.hdf5
Epoch 00003: val_loss improved from 2.69877 to 2.67814, saving model to tensorflow/tmp/weigths_movie_barcode_classifier.hdf5
Epoch 00004: val_loss improved from 2.67814 to 2.65133, saving model to tensorflow/tmp/weigths_movie_barcode_classifier.hdf5
Epoch 00005: val_loss improved from 2.65133 to 2.61642, saving model to tensorflow/tmp/weigths_movie_barcode_classifier.hdf5
Epoch 00006: val_loss improved from 2.61642 to 2.57050, saving model to tensorflow/tmp/weigths_movie_barcode_classifier.hdf5
Epoch 00007: val_loss improved from 2.57050 to 2.50831, saving 

# Load model from file

In [12]:
#from keras.models import load_model
#model = load_model('tensorflow/model/model_movie_barcode_soundtracks.h5')
#model.load_weights('tensorflow/model/model_movie_barcode_weigths_soundtracks.h5')

## Evaluate model on test

In [13]:
# finally, evaluate the model using the withheld test dataset
#score, accuracy = model.evaluate(test_x, test_y, batch_size=80)
score, accuracy = model.evaluate(test_x, test_y, batch_size=80)
print("\nAccuracy = {:.2f}".format(accuracy))

160/166 [===========================>..] - ETA: 0s
Accuracy = 0.24


In [14]:
predictions = model.predict_classes(test_x)
print(predictions)
print("Predictions {}".format(predictions))

160/166 [===========================>..] - ETA: 0s[12  1 12  1  1  1  1  1  1 12  1  1  1  1  1 12  1 12  1  1 12  1  1  1  1
 12  1 12  1  1  1  1  1 12 12  1 12  1 12  1  1  1 12  1  1  1  1  1  1  1
 12  1  1  1 12  1  1 12  1  1 12 12 12  1 12  1  1  1  1 12  1  1 12 12 12
 12  1  1  1 12  1  1  1 12 12  1 12  1  1  1  1 12 12 12  1  1 12  1 12 12
  1 12  1  1  1  1  1  1  1  1  1  1  1  1  1 12 12  1  1  1 12  1  1  1  1
  1  1 12  1 12  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  1 12  1  1
  1  1  1  1 12  1  1 12  1  1  1  1  1  1  1  1]
Predictions [12  1 12  1  1  1  1  1  1 12  1  1  1  1  1 12  1 12  1  1 12  1  1  1  1
 12  1 12  1  1  1  1  1 12 12  1 12  1 12  1  1  1 12  1  1  1  1  1  1  1
 12  1  1  1 12  1  1 12  1  1 12 12 12  1 12  1  1  1  1 12  1  1 12 12 12
 12 

In [15]:
from sklearn.metrics import confusion_matrix
y_pred = predictions
print(test_y[:,0])
print(le.classes_)
y_true = test_y[:,0]
confusion_matrix(y_true, y_pred)

[ 1  2  1 12 11  0  1 12 12  9  1 12  7  2 12  3  1  9 12 10  2 12  1  2  9
 12 12 11 12  9  2  9  8  1  7 12  3 12 12  2  2  9  1  1  2  1 12 12  9  1
 12  9  0  7  7  1  2  1  3 12  3 12  2  2  9  9 12 12  1 12 12  1  2  3 12
  1 12  2  9  2  2 12  2  3 12  7 12 12  7  1  9  1 12  1  1  9 12  2 12 12
  1  1  1  1 12 12 12  7  2  9 12  7 12  9  9 13  7 12 12  2  2  1  2 12 12
  2 13 12 12 12  2  1 12  9 12  9  2 13  1  2  2 12  6 12  7 12  9 12  2  1
  1 12  2  9  2  3  2 12  1 12  9  1  3  1  9  3]
['Drama' 'action' 'adventure' 'comedy' 'crime' 'dance' 'documentary'
 'drama' 'gangster' 'horror' 'musical' 'romance' 'scifi' 'war' 'western']


array([[ 0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 23,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0],
       [ 0, 24,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0],
       [ 0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 19,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
       [ 0, 36,  0,  0,  0,  0,  0,  0,  0,  0, 17,  0],
       [ 0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0]])

### Predict one sample

In [16]:
def predict(moviebarcode_path, width, heigth):
    #TODO
    pass

# Check overall Accuracy (Test Independent Dataset)

In [17]:
parent_dir = "movie_genres"
sub_dirs = ["trailers_test"]
#TODO check_overall_accuracy(parent_dir,sub_dirs, bands, frames)